### LIBRERÍAS

In [2]:
# Librerías básicas
import numpy as np
import pandas as pd

# Librerías de Preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn import metrics 


#Librerís de Modelado y evaluación
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pathlib import Path


### DATOS SÁTELITALES

In [3]:

satelitales = pd.read_csv('../Data/Boyaca_NDVI_EVI_SoilMoisture_MENSUAL_2005_2025_CLEAN.csv')
satelitales



,EVI,NDVI,fecha,year,month,yyyymm
0,0.381304,0.688825,2005-01-01,2005,1,2005-01
1,0.362783,0.668144,2005-02-01,2005,2,2005-02
2,0.379677,0.632942,2005-03-01,2005,3,2005-03
3,0.435677,0.667824,2005-04-01,2005,4,2005-04
4,0.405432,0.695091,2005-05-01,2005,5,2005-05
...,...,...,...,...,...,...
243,0.419982,0.695208,2025-04-01,2025,4,2025-04
244,0.434576,0.731797,2025-05-01,2025,5,2025-05
245,0.405199,0.743507,2025-06-01,2025,6,2025-06
246,0.453621,0.732904,2025-07-01,2025,7,2025-07


In [4]:
satelitales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   EVI     248 non-null    float64
 1   NDVI    248 non-null    float64
 2   fecha   248 non-null    object 
 3   year    248 non-null    int64  
 4   month   248 non-null    int64  
 5   yyyymm  248 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 11.8+ KB


In [5]:
satelitales.describe()

,EVI,NDVI,year,month
count,248.000000,248.000000,248.000000,248.000000
mean,0.415884,0.690760,2014.838710,6.435484
std,0.030513,0.031170,5.980247,3.445920
min,0.322249,0.547600,2005.000000,1.000000
25%,0.396819,0.674575,2010.000000,3.000000
50%,0.419184,0.697605,2015.000000,6.000000
75%,0.437864,0.712912,2020.000000,9.000000
max,0.499423,0.746885,2025.000000,12.000000


### ACOTACIÓN TIEMPO Y CORRECIÓN FECHA

In [6]:


# 1) Copia y normaliza nombres
df_ndvi = satelitales.copy()
df_ndvi.columns = df_ndvi.columns.str.strip().str.lower()

# 2) Detecta columna de fecha y pásala a datetime
fecha_col = 'fecha' if 'fecha' in df_ndvi.columns else [c for c in df_ndvi.columns if 'time' in c or 'fecha' in c][0]
df_ndvi[fecha_col] = pd.to_datetime(df_ndvi[fecha_col], errors='coerce')

# 3) Filtra 2012–2019
df_ndvi = df_ndvi[(df_ndvi[fecha_col].dt.year >= 2012) & (df_ndvi[fecha_col].dt.year <= 2019)]

# 4) Quédate solo con fecha + NDVI/EVI (en cualquier variante de nombre)
idx_cols = [c for c in df_ndvi.columns if c == 'ndvi' or c == 'evi' or c.startswith('ndvi') or c.startswith('evi')]
df_ndvi = df_ndvi[[fecha_col] + idx_cols].sort_values(fecha_col).reset_index(drop=True)
# (Opcional) Si hubiera filas repetidas por fecha:
# df_ndvi = df_ndvi.drop_duplicates(subset=[fecha_col])

# 5) Campos auxiliares
df_ndvi['year'] = df_ndvi[fecha_col].dt.year
df_ndvi['month'] = df_ndvi[fecha_col].dt.month
df_ndvi['yyyymm'] = df_ndvi[fecha_col].dt.strftime('%Y-%m')

print("NDVI/EVI listo:", df_ndvi.shape)
display(df_ndvi.head())


NDVI/EVI listo: (96, 6)


,fecha,evi,ndvi,year,month,yyyymm
0,2012-01-01,0.398118,0.699554,2012,1,2012-01
1,2012-02-01,0.390345,0.671628,2012,2,2012-02
2,2012-03-01,0.376592,0.643635,2012,3,2012-03
3,2012-04-01,0.451125,0.697795,2012,4,2012-04
4,2012-05-01,0.447870,0.697231,2012,5,2012-05


### RENDIMIENTOS CULTIVO CAFÉ

In [7]:
rendimientos = pd.read_excel('../Data/Producción y Rendimiento Mensual en Boyaca_12-19.xlsx')
rendimientos


,Fecha,Año,Mes,Boyaca_Prod_toneladas,AreaCultivada_ha,Rendimiento_t-ha
0,2012-01-01,2012,1,91.463307,4791.05,0.027630
1,2012-02-01,2012,2,97.617847,4791.05,0.029489
2,2012-03-01,2012,3,98.472645,4791.05,0.029747
3,2012-04-01,2012,4,99.156482,4791.05,0.029954
4,2012-05-01,2012,5,117.791063,4791.05,0.035583
...,...,...,...,...,...,...
91,2019-08-01,2019,8,748.984354,12312.95,0.067459
92,2019-09-01,2019,9,728.235011,12312.95,0.065591
93,2019-10-01,2019,10,916.317766,12312.95,0.082531
94,2019-11-01,2019,11,1008.016476,12312.95,0.090790


In [8]:
rendimientos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Fecha                  96 non-null     datetime64[ns]
 1   Año                    96 non-null     int64         
 2   Mes                    96 non-null     int64         
 3   Boyaca_Prod_toneladas  96 non-null     float64       
 4   AreaCultivada_ha       96 non-null     float64       
 5   Rendimiento_t-ha       96 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 4.6 KB


In [9]:
rendimientos.describe()

,Fecha,Año,Mes,Boyaca_Prod_toneladas,AreaCultivada_ha,Rendimiento_t-ha
count,96,96.000000,96.000000,96.000000,96.000000,96.000000
mean,2015-12-16 11:00:00,2015.500000,6.500000,471.616979,7602.197500,0.067611
min,2012-01-01 00:00:00,2012.000000,1.000000,88.727956,4232.810000,0.026803
25%,2013-12-24 06:00:00,2013.750000,3.750000,219.163346,5139.477500,0.053199
50%,2015-12-16 12:00:00,2015.500000,6.500000,456.275407,6723.035000,0.068274
75%,2017-12-08 18:00:00,2017.250000,9.250000,721.758222,10195.470000,0.080680
max,2019-12-01 00:00:00,2019.000000,12.000000,1124.480531,12312.950000,0.115797
std,NaN,2.303316,3.470174,274.341289,2861.117089,0.020592


### DATOS ANUALES - CONVERSIÓN A MENSUALES

In [12]:
# ============================================
# RENDIMIENTOS REALES DESDE EXCEL (BOYACÁ)
# ============================================

# 1) Partimos del DataFrame `rendimientos` leído del Excel
#   Columnas reales del archivo:
#   ['Fecha', 'Año', 'Mes', 'Boyaca_Prod_toneladas',
#    'AreaCultivada_ha', 'Rendimiento_t-ha']

# Renombrar columnas para que coincidan con las usadas en el notebook viejo
rend = rendimientos.rename(columns={
    "Fecha": "fecha",
    "Año": "anio",
    "Mes": "month",
    "Rendimiento_t-ha": "rendimiento_t_ha"
}).copy()

# 2) Campos adicionales que antes se fabricaban
rend["departamento"] = "Boyacá"
rend["fecha"] = pd.to_datetime(rend["fecha"], errors="coerce")
rend["yyyymm"] = rend["fecha"].dt.strftime("%Y-%m")

# 3) Este es el DataFrame que antes salía del código simulado
df_rend_boyaca = rend[[
    "departamento",
    "fecha",
    "anio",
    "month",
    "yyyymm",
    "rendimiento_t_ha"
]].copy()

# 4) Comprobación rápida de que cuadra por año
df_check = df_rend_boyaca.groupby("anio")["rendimiento_t_ha"].sum().round(3)
print("🔎 Comprobación de sumas anuales (t/ha):")
print(df_check)

print("\n👀 Primeras filas de df_rend_boyaca:")
display(df_rend_boyaca.head(24))
# Datos anuales de rendimiento de café en Boyacá (t/ha) - DIVISIÓN MENSUAL

# --- Datos anuales de Boyacá (t/ha) ---
rend_anual = {
    2012: 0.5,
    2013: 0.9,
    2014: 0.8,
    2015: 0.6,
    2016: 1.0,
    2017: 1.1,
    2018: 1.0,
    2019: 0.9
}

# --- Distribución estacional mensual ---
# Normalizada para que sume 1 (po



🔎 Comprobación de sumas anuales (t/ha):
anio
2012    0.400
2013    0.862
2014    0.722
2015    0.599
2016    0.986
2017    1.060
2018    0.972
2019    0.889
Name: rendimiento_t_ha, dtype: float64

👀 Primeras filas de df_rend_boyaca:


,departamento,fecha,anio,month,yyyymm,rendimiento_t_ha
0,Boyacá,2012-01-01,2012,1,2012-01,0.027630
1,Boyacá,2012-02-01,2012,2,2012-02,0.029489
2,Boyacá,2012-03-01,2012,3,2012-03,0.029747
3,Boyacá,2012-04-01,2012,4,2012-04,0.029954
4,Boyacá,2012-05-01,2012,5,2012-05,0.035583
5,Boyacá,2012-06-01,2012,6,2012-06,0.036874
6,Boyacá,2012-07-01,2012,7,2012-07,0.034498
7,Boyacá,2012-08-01,2012,8,2012-08,0.029179
8,Boyacá,2012-09-01,2012,9,2012-09,0.026803
9,Boyacá,2012-10-01,2012,10,2012-10,0.033724



Cada año se descompone en 12 meses, respetando la estacionalidad típica del café:

Mitaca: abril–junio con picos leves.

Principal: octubre–diciembre con picos fuertes.

Enero–marzo y julio–septiembre con actividad moderada/baja.

### SIMULACIÓN VARIABLES CLIMÁTICAS

In [21]:
import pandas as pd
import numpy as np

# =========================
# 1. ERA5 mensual (Tmax, Tmin, humedad)
# =========================
df_era5 = pd.read_csv("../Data/Clima_Boyaca_ERA5_2012_2019.csv")
df_era5.columns = df_era5.columns.str.strip()

# Columnas según tu archivo ERA5
col_tmax  = "Tmax_C"
col_tmin  = "Tmin_C"
col_hum   = "RH_min"
col_year  = "year"
col_month = "month"

# Fecha correcta: año + mes reales
df_era5["fecha"] = pd.to_datetime(
    dict(year=df_era5[col_year], month=df_era5[col_month], day=1)
)

df_era5 = df_era5[["fecha", col_tmax, col_tmin, col_hum]].copy()
df_era5.rename(columns={
    col_tmax: "TempMax",
    col_tmin: "TempMin",
    col_hum: "HumedadRelativa"
}, inplace=True)

print("ERA5 mensual:", df_era5.shape)
print(df_era5.head())


# =========================
# 2. Precipitación mensual (codificada en el día)
# =========================
df_p = pd.read_csv("../Data/precipitacion_boy.csv")
df_p.columns = df_p.columns.str.strip()

col_fecha_p = df_p.columns[0]       # "Fecha"
col_pp      = df_p.columns[1]       # "P_promedio_mm"

df_p[col_fecha_p] = pd.to_datetime(df_p[col_fecha_p], errors="coerce")

# Aquí está el truco:
# - el AÑO viene bien en la fecha
# - el MES está en el DÍA
df_p["anio"] = df_p[col_fecha_p].dt.year
df_p["mes"]  = df_p[col_fecha_p].dt.day  # 1..12 = enero..diciembre

# Construimos la fecha mensual verdadera
df_p["fecha"] = pd.to_datetime(dict(year=df_p["anio"], month=df_p["mes"], day=1))

# Como ya es una fila por mes, NO agregamos nada, solo renombramos
df_p_mensual = df_p[["fecha", col_pp]].copy()
df_p_mensual.rename(columns={col_pp: "Precipitacion"}, inplace=True)

print("Precipitación mensual:", df_p_mensual.shape)
print(df_p_mensual.head(15))  # debería mostrar 2012-01-01, 2012-02-01, ..., 2013-01-01, etc.


# =========================
# 3. MERGE FINAL
# =========================
df_clima = df_era5.merge(df_p_mensual, on="fecha", how="inner")

print("df_clima final:", df_clima.shape)
display(df_clima.head(12))  # primeros 12 meses (todo 2012)



ERA5 mensual: (96, 4)
       fecha    TempMax    TempMin  HumedadRelativa
0 2012-01-01  18.751319  11.248684        91.408108
1 2012-02-01  19.348317  10.918546        88.014541
2 2012-03-01  18.907482  11.580643        93.213246
3 2012-04-01  18.589837  11.784516        95.391357
4 2012-05-01  18.641664  11.675996        94.024900
Precipitación mensual: (96, 2)
        fecha  Precipitacion
0  2012-01-01         89.925
1  2012-02-01         64.750
2  2012-03-01        224.175
3  2012-04-01        272.475
4  2012-05-01        204.925
5  2012-06-01        134.600
6  2012-07-01        164.600
7  2012-08-01        180.350
8  2012-09-01         82.925
9  2012-10-01        201.500
10 2012-11-01        159.725
11 2012-12-01         46.475
12 2013-01-01         38.200
13 2013-02-01         79.775
14 2013-03-01        125.750
df_clima final: (96, 5)


,fecha,TempMax,TempMin,HumedadRelativa,Precipitacion
0,2012-01-01,18.751319,11.248684,91.408108,89.925
1,2012-02-01,19.348317,10.918546,88.014541,64.750
2,2012-03-01,18.907482,11.580643,93.213246,224.175
3,2012-04-01,18.589837,11.784516,95.391357,272.475
4,2012-05-01,18.641664,11.675996,94.024900,204.925
5,2012-06-01,18.776218,10.851417,91.551220,134.600
6,2012-07-01,18.570666,11.154045,91.144814,164.600
7,2012-08-01,18.762646,11.106604,90.929191,180.350
8,2012-09-01,19.607542,10.748671,90.290405,82.925
9,2012-10-01,19.201556,11.361715,93.928322,201.500


### INTEGRACIÓN DATAFRAMES

In [12]:
#INTEGRACIÓN 

# yyyymm de cada fuente
df_ndvi["yyyymm"]       = df_ndvi[fecha_col].dt.strftime("%Y-%m")
df_rend_boyaca["yyyymm"]= df_rend_boyaca["fecha"].dt.strftime("%Y-%m")
df_clima["yyyymm"]      = df_clima["fecha"].dt.strftime("%Y-%m")

# Merge por yyyymm
df_modelo = (
    df_ndvi.merge(df_rend_boyaca[["yyyymm","rendimiento_t_ha"]], on="yyyymm", how="inner")
           .merge(df_clima.drop(columns=["fecha"], errors="ignore"), on="yyyymm", how="left")
           .rename(columns={"rendimiento_t_ha":"rendimiento"})
           .assign(fecha=lambda d: pd.to_datetime(d["yyyymm"]))
           .assign(anio=lambda d: d["fecha"].dt.year, month=lambda d: d["fecha"].dt.month)
           .sort_values("fecha", kind="stable")
           .reset_index(drop=True)
)

# (opcional) Reordenar para visualización rápida
cols = ["fecha","yyyymm","ndvi", "evi", "rendimiento","Precipitacion","TempMax","TempMin","HumedadRelativa"]
df_modelo = df_modelo[[c for c in cols if c in df_modelo.columns]]

# Mostrar tabla limpia
print("Dataset mensual para modelado:", df_modelo.shape)
display(df_modelo.head(12))

Dataset mensual para modelado: (96, 9)


,fecha,yyyymm,ndvi,evi,rendimiento,Precipitacion,TempMax,TempMin,HumedadRelativa
0,2012-01-01,2012-01,0.699554,0.398118,0.020,179.868566,25.444180,15.256912,79.066254
1,2012-02-01,2012-02,0.671628,0.390345,0.025,154.469428,25.391583,13.505113,76.565570
2,2012-03-01,2012-03,0.643635,0.376592,0.030,185.907542,25.007670,15.207817,79.344953
3,2012-04-01,2012-04,0.697795,0.451125,0.040,220.921194,24.648119,15.462381,79.928520
4,2012-05-01,2012-05,0.697231,0.447870,0.050,150.633865,22.876944,13.939371,86.724130
5,2012-06-01,2012-06,0.710103,0.415739,0.060,150.634522,24.369032,15.184470,85.072678
6,2012-07-01,2012-07,0.704582,0.441602,0.040,223.168513,24.485928,15.069850,83.127334
7,2012-08-01,2012-08,0.700551,0.423252,0.030,190.697389,23.796584,13.628436,82.734935
8,2012-09-01,2012-09,0.691045,0.397087,0.035,141.221025,24.758071,15.429345,83.666514
9,2012-10-01,2012-10,0.702355,0.421132,0.050,181.702402,25.606076,15.672941,78.497593


### PREPROCESAMIENTO

In [13]:
# PREPROCESAMIENTO SIMPLE

# 1) Copia y asegura columnas de tiempo
datos = df_modelo.copy()

# fecha ↔ yyyymm
if "fecha" not in datos.columns:
    if "yyyymm" in datos.columns:
        datos["fecha"] = pd.to_datetime(datos["yyyymm"], format="%Y-%m", errors="coerce")
    else:
        raise ValueError("Falta 'fecha' o 'yyyymm' en df_modelo.")
datos["mes"]  = datos["fecha"].dt.month
datos["anio"] = datos["fecha"].dt.year

# 2) Estandariza nombres NDVI/EVI y clima (tolerante a mayúsculas/minúsculas)
renombres = {}
for c in datos.columns:
    cl = c.strip().lower()
    if cl.startswith("ndvi"): renombres[c] = "NDVI"
    if cl.startswith("evi"):  renombres[c] = "EVI"
    if cl in {"precipitacion"}: renombres[c] = "Precipitacion"
    if cl in {"tmax","tempmax"}: renombres[c] = "TempMax"
    if cl in {"tmin","tempmin"}: renombres[c] = "TempMin"
    if cl in {"humedad","humedadrelativa"}: renombres[c] = "HumedadRelativa"
datos = datos.rename(columns=renombres)

# 3) Estacionalidad mensual (seno/coseno)
datos["mes_sin"] = np.sin(2*np.pi*datos["mes"]/12)
datos["mes_cos"] = np.cos(2*np.pi*datos["mes"]/12)

# 4) Selección de variables (toma solo las que existan)
variables_base = ["NDVI","EVI","Precipitacion","TempMax","TempMin","HumedadRelativa"]
variables_modelo = [v for v in variables_base if v in datos.columns] + ["mes_sin","mes_cos"]

X = datos[variables_modelo].apply(pd.to_numeric, errors="coerce")   # predictores
y = pd.to_numeric(datos["rendimiento"], errors="coerce")            # objetivo

# 5) División temporal y transformación (imputar + escalar)
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(
    X, y, test_size=0.20, shuffle=False
)

prepro = Pipeline([
    ("imputar", SimpleImputer(strategy="median")),
    ("escalar", StandardScaler())
])

X_entrena_prep = prepro.fit_transform(X_entrena)
X_prueba_prep  = prepro.transform(X_prueba)

print("Variables usadas:", variables_modelo)
print("Variable objetivo:", "rendimiento")
print("Train:", X_entrena_prep.shape, " | Test:", X_prueba_prep.shape)


Variables usadas: ['NDVI', 'EVI', 'Precipitacion', 'TempMax', 'TempMin', 'HumedadRelativa', 'mes_sin', 'mes_cos']
Variable objetivo: rendimiento
Train: (76, 8)  | Test: (20, 8)


## 🌎 ¿Por qué usamos `mes_sin` y `mes_cos`?

En datos **mensuales** como los de la caficultura, el **mes del año influye fuertemente** en el rendimiento (por lluvias, floración, cosecha, etc.).  
Sin embargo, el número del mes (`1–12`) es una **variable cíclica**, no lineal.

### 🧩 Problema
Si usas el número del mes directamente, el modelo interpretará que:

> Diciembre (12) está muy lejos de Enero (1),

cuando en realidad **son adyacentes** en el ciclo anual.

### 🧭 Solución: *codificación cíclica*

Transformamos el número del mes en **coordenadas circulares** usando funciones seno y coseno.  
Así, los meses se representan sobre un círculo, donde diciembre y enero “se tocan”.

### 📐 Fórmulas

\[
\text{mes\_sin} = \sin\left(2\pi \times \frac{\text{mes}}{12}\right)
\]

\[
\text{mes\_cos} = \cos\left(2\pi \times \frac{\text{mes}}{12}\right)
\]

### 🌀 Ejemplos

| Mes | mes_sin | mes_cos | Interpretación |
|-----|----------|----------|----------------|
| Enero | ≈ 0.5 | ≈ 0.87 | Inicio de año |
| Julio | ≈ -1 | ≈ 0 | Mitad del año |
| Diciembre | ≈ 0 | ≈ 1 | Fin del año |

### 💡 Beneficio

El modelo aprende **la estacionalidad continua**, es decir, los cambios **graduales entre meses**,  
sin los saltos artificiales que ocurren si se usan números enteros.


### MODELADO Y SIMULACIÓN

In [40]:
#  MODELADO Y EVALUACIÓN 

# Modelos a probar
modelos = {
    "Regresión Lineal": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42)
}

resultados = []

# Entrenar y evaluar cada modelo
for nombre, modelo in modelos.items():
    modelo.fit(X_entrena_prep, y_entrena)
    y_pred = modelo.predict(X_prueba_prep)
    
    mae = mean_absolute_error(y_prueba, y_pred)
    rmse = np.sqrt(mean_squared_error(y_prueba, y_pred))
    r2 = r2_score(y_prueba, y_pred)
    
    resultados.append([nombre, mae, rmse, r2])

# Tabla de resultados
df_resultados = pd.DataFrame(resultados, columns=["Modelo", "MAE", "RMSE", "R2"]).sort_values("R2", ascending=False)

print("📊 Resultados del modelado:")
display(df_resultados)

# Mejor modelo
mejor_modelo = df_resultados.iloc[0, 0]
print(f"✅ Mejor modelo: {mejor_modelo}")


📊 Resultados del modelado:


,Modelo,MAE,RMSE,R2
1,Random Forest,0.012968,0.018749,0.524479
0,Regresión Lineal,0.016795,0.022187,0.334078
2,Gradient Boosting,0.016886,0.022566,0.311165


✅ Mejor modelo: Random Forest


### EXPORTO DF_MODELO PARA PODER USAR EN PIPELINE

In [41]:
from pathlib import Path

CACHE = Path("artifacts/_cache")
CACHE.mkdir(parents=True, exist_ok=True)

# Formato preferido (conserva tipos y es más compacto)
df_modelo.to_parquet(CACHE / "df_modelo.parquet", index=False)

# Opcional: también como CSV por si quieres verlo a ojo
df_modelo.to_csv(CACHE / "df_modelo.csv", index=False, encoding="utf-8-sig")

print("✅ Exportado df_modelo a artifacts/_cache/")


✅ Exportado df_modelo a artifacts/_cache/
